Лицензия MIT

© Алексей Александрович Щербаков, 2024

# Лекция 5.3. Глубокое обучение в задачах оптики и фотоники

Глубокое обучение основано на использовании глубоких нейронных сетей (DNN) для решения как прямых, так и обратных задач. В отличие от традиционных аналитических подходов, глубокое обучение оказывается полезно, когда знаний о рассматриваемой системе недостаточно для применения более традиционных оптимизационных методов, или когда существует слишком много степеней свободы, что существенно затрудняют прогнозирование поведения системы. В таких сценариях «черного ящика» сети можно обучать, используя только входные и выходные данные, без необходимости знания и понимания внутренней работы системы. С одной стороны, такой подход идет несколько в разрез с традиционным научным и инженерным методом, но бывают ситуации, когда только он позволяет получать требуемые решения. Ниже дан короткий обзор общих понятий и подходов глубокого обучения, а затем приведены примеры его использования в задачах оптики и фотоники.

## Нейронные сети

Основной составляющей нейронных сетей является так называемый искусственный нейрон, который реализуется с помощью векторной математической функции, представляющей собой нелинейное преобразование взвешенной суммы компонентов входного вектора, к которой применяется нелинейная "функция активации" $\varphi$:
$$ f:\;\boldsymbol{x}\rightarrow a = \boldsymbol{w}^T\boldsymbol{x}+b \rightarrow y=\varphi(a) $$
Предполагается, что функция активации непрерывно дифференцируема за исключением, может быть, конечного числа точек, и, как правило, представляет собой сигмоиду, ReLU или гиперболический тангенс. Нейрон является аналогом логистической регрессии.

<figure> <center>
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/c/c6/Artificial_neuron_structure.svg/1920px-Artificial_neuron_structure.svg.png" alt="Artificial Neuron, Wiki" width="500">
<br><caption>Визуализации функции искусственного нейрона (Википедия)</caption>
</center> </figure>

Нейронная сеть - некоторый набор искусственных нейронов, соединенных связями, организованных с целью решения некоторой задачи. Сеть может быть представлена в виде графа, узлы которого - нейроны, а ребра показывают связи между ними. Связи показывают, как связаны выходы одних функций, реализующих нейроны, со входами других нейронов. Часто работают со слоями нейронных сетей и связями между слоями. Вся совокупность нейронов сети, их функций активации и связей между ними, а также типов слоёв, называется архитектурой сети. Выделяют входной слой, принимающий исходный вектор параметров (или входной сигнал), выходной слой, выдающий вектор - результат работы сети, а также, возможно, дополнительные скрытые слои. Если скрытых слоёв больше одного, сеть называется глубокой.

<figure> <center>
<img src="https://media.geeksforgeeks.org/wp-content/uploads/20240719175629/Input-Layer-in-an-ANN.webp" alt="Artificial Neuron, Wiki" width="500">
<br><caption><a href="https://www.geeksforgeeks.org/deep-learning/layers-in-artificial-neural-networks-ann/">Пример нейронной сети со скрытыми слоями</a></p></caption>
</center> </figure>

Пример "зоопарка" архитектур нейронных сетей с сайта [The Asimov Institute](https://www.asimovinstitute.org) приведен на рисунке ниже
<figure> <center>
<img src="https://www.asimovinstitute.org/wp-content/uploads/2019/04/NeuralNetworkZoo20042019.png" alt="The Neural Network Zoo" width="500">
</center> </figure>

Скрытые слои нейронов в сети могут быть различных типов - полносвязные, сверточные, рекуррентные, выпадающие (dropout layer), пулинговые (pooling layer) и слои пакетной нормализации (batch normalization layer). В полносвязном слое каждый нейрон связан со всеми нейронами в предыдущем и последующем слоях, а веса нейронов являются параметрами сети. Сверточный слой представляет собой применение операции свертки к выходам предыдущего слоя, так что ядро свертки является параметром сети. В рекуррентных слоях используются циклы обратной связи, как правило, с запаздыванием. Выпадающие слои используются для регуляризации работы сети и избежания переобучения, так что при обучении сети (нахождении оптимальных параметров) нейроны такого слоя случайным образом исключаются из работы. Пулинговые слои используются для снижения размерности данных, например, путем вычисления максимального или среднего значения от блоков входных данных. Слои пакетной нормализации используются для ускорения обучения, в них происходит нормировка выхода предыдущего слоя путем вычитания среднего по пакету и деления на стандартное отклонение по пакету.

Задача обучения сводится к нахождению весов и сдвигов функций нейронов, ядер светрок и т.п. Обучение основано на алгоритме обратного распространения ошибки - вычислению градиента по всем параметрам сети с целью оптимизации значений параметров на обучающей выборке с известными результатами целевой функции сети. При инициализации сети веса, характеризующие функции нейронов задаются, случайным образом. Затем в итеративном алгоритме оптимизации веса корректируются так, чтобы входные и выходные данные сети соответствовали значениям в так называемом обучающем наборе. Эта задача оптимизации может быть сформиулирована как задача минимазации некоторого функционала - функции потерь, которая количественно определяет разницу между выходными данными сети и заданными значениями из обучающего набора. Например, в задачах регрессии - предсказания выходных числовых значений некоторой непрерывной величины по заданным входным данным - функция потерь часто выбирается как среднеквадратичное отклонение
$$ L(y,\hat{y}) = \frac{1}{N} \sum_{n=1}^{N} (y_n-\hat{y}_n)^2 $$
где $y_n$ - выходные данные обучающей выборки, а $\hat{y}_n$ - выходные значения текущего состояния сети. Значение $N$ может быть как равно всему объему обучающей выборки, так и быть меньше него. Во втором случае из всей выборки случайным образом выбирается эталонный набор для обучения.

Большинство задач в области машинного и глубокого обучения, в применении к физике, в частности, сводятся к так называемым генеративной и дискриминативной моделям. Дискриминативные сети нужны для решения задач регрессии и классификации. В них происходит интерполяция отношений в обучающих данных. Отображение входных параметров на выходные является однозначной функцией, представляющей собой отображение многомерного на одномерное или также многомерное пространство векторов. В терминах вероятностей в рамках дискриминативного моделирования для некоторого объекта $x$ и характеристики $y$ происходит восстановление функции условной вероятности $p(y|x)$. Таким образом, дискриминативные нейронные сети могут служить суррогатными физическими моделями, которые решают прямую задачу, например, рассеяния или дифракции. По сравнению со строгими численными моделями физических явлений обученная дискриминативная сеть может давать оценку решения прямой задачи на порядки быстрее классических строгих методов.

Генеративные модели служат для генерации объектов некотрого класса по заданным характеристикам. При генеративном моделировании восстанавливаются вероятность $p(x)$ или $p(x|y)$. В результате обучения генеративной модели получается некторая функция, выдающая объекты заданного класса. Примерами генеративных моделей являются генеративные состязательные нейронные сети (GAN), диффузионные модели, вариационные автокодировщики. В этих моделях объект является функцией некоторого случайного вектора, компоненты которого имеют простое вероятностное распределение (например, равномерное или нормальное) $x=G(z)$, где $G$ - функция генератора, а $z$ - вектор так называемого латентного пространства. Задание направлений в латентном пространстве позволяет плавно менять различные характеристики объекта.

## Дискриминативное моделирование оптических и фотонных устройств

Бурное развитие методов искусственного интеллекта, которе началось в конце 2000-х годов всвязи с развитием технологий производства видеокарт, довольно быстро привело к активному использованию этих методов в ествественнонаучных задача, в частности в оптике и фотонике. На рисунке ниже представлены некоторые примеры применений различных типов нейронных сетей для суррогатного моделирования при решении прямых задач оптики и фотоники. Для обучения сетей генерировались наборы данных из десятков тысяч образцов с помощью строгого моделирования.

<figure> <center>
<img src="https://media.springernature.com/full/springer-static/image/art%3A10.1038%2Fs41578-020-00260-1/MediaObjects/41578_2020_260_Fig3_HTML.png?as=webp" width="1000">
<br><caption><a href="https://www.nature.com/articles/s41578-020-00260-1">Примеры суррогатных моделей в оптике и фотонике. (a) временная шкала появления различных приложений, (b) аппроксимация спектров рассеяния многослойных сферических частиц с помощью полносвязной сети, (c) расчет пространственной поляризации наноструктуры при возбуждении электромагнитной волной с помощью сверточной сети, (d) модель расчета спектра связанных кольцевых резонаторов с помощью графовой сети</a></p></caption>
</center> </figure>

Для решения обратных задач было предложено несколько различных подоходов, использующих нейронные сети. Одна из возможностей - использонование алгоритма обратного распространения ошибок для коррекции входных данных, которые, например, задают геометрию рассчитываемого оптического устройства, для обученной нейронной сети с фиксированными весами.

<figure> <center>
<img src="https://media.springernature.com/full/springer-static/image/art%3A10.1038%2Fs41578-020-00260-1/MediaObjects/41578_2020_260_Fig4_HTML.png?as=webp" width="1000">
<br><caption><a href="https://www.nature.com/articles/s41578-020-00260-1">Примеры решения обратной задачи с помощью нейронных сетей. (a,b) применение пред-натренированной сети и алгоритма обратного распространения ошибок (c) пример использования прямой суррогатной модели в связке с генетическим алгоритмом, (d-e) применение суррготной прямой модели для обучения сети для решения обратной задачи</a></p></caption>
</center> </figure>

Другой подход - использование дискриминативных моделей как прямых решателей в комбинации с классическими оптимизационными алгоритмами, часть из которых упоминалась в предыдущих лекциях. Пример подобного похода показан на рисунке выше. Также предлагалось использование комбинаций нейронных сетей для решения сначала прямой, а затем обратной задачи в тех случаях, когда использование строгих прямых и классических оптимизационных методов для решения обратных задач оказывается чрезмерно трудозатратным.

## Генеративное моделирование оптических и фотонных устройств

Особенности генеративных моделей позволяют по-разному использовать их при проектировании оптических устройств. Первый способ состоит в обучении безусловнрй сети на наборе устройств, которые представляют собой небольшую выборку целевого подмножества пространства проектирования. Обучающие наборы могут представлять устройства различных геометрий, которые в совокупности обладают набором требуемых оптических откликов. В этом случае обученная сеть будет генерировать распределения устройств, которые более полно заполняют подпространство проектирования. Обучающие наборы также могут включать варианты одного и того же типа устройств, так что обученная сеть будет генерировать больше геометрических вариаций этого типа устройств, некоторые из которых могут работать лучше, чем в устройства в обучающем наборе.

Вторая стратегия заключается в обучении условной сети на наборах высокопроизводительных оптимизированных устройств. Если обучающий набор состоит из устройств, работающих с определенными дискретными значениями условных меток, обученная сеть сможет обобщать и создавать макеты устройств по всему непрерывному спектру значений меток. Эта способность генерировать устройства со значениями условных меток, интерполированными из обучающего набора, аналогична регрессии с дискриминативными сетями.

Третья стратегия заключается в том, чтобы изначально обучить либо условную, либо безусловную генеративную сеть, а затем использовать обычные методы оптимизации на пространстве латентных параметров. Здесь можно провести параллели с подходом, в котором дискриминационная модель используется в качестве суррогатного электромагнитного решателя в сочетании с обычными методами оптимизации. Принципиальное отличие здесь заключается в том, что генеративные сети обеспечивают больший контроль над пространством поиска потенциальных оптимальных устройств.

<figure> <center>
<img src="https://media.springernature.com/full/springer-static/image/art%3A10.1038%2Fs41578-020-00260-1/MediaObjects/41578_2020_260_Fig6_HTML.png?as=webp" width="1000">
<br><caption><a href="https://www.nature.com/articles/s41578-020-00260-1">Иллюстрация путей применения генеративного ИИ для решения обратных задач оптики и фотоники: (a) безусловная сеть обучается на нектором пространстве устройств и используется для поиска похожих устройств; (b) | условная сеть используется для генерации распределения устройств с помощью условных меток параметров, интерполированных из обучающего набора - например, сеть изучает обучается на устройствах, функционирующих на длинах волн 800 и 1000 нм, и может интерполировать распределение устройств, работающих на 900 нм; (c) использование классических методов оптимизации для поиска оптимального пространства на множестве латентных параметров.</a></p></caption>
</center> </figure>

## Основанные на физике нейронные сети (PINN - physics-informed neural networks)

Основанные на физике нейронные сети - это класс моделей машинного обучения, которые интегрируют физические законы (выраженные в виде дифференциальных уравнений) в процесс обучения нейронной сети. В отличие от традиционных нейронных сетей, которые полагаются исключительно на данные, PINN используют известную физику для улучшения, обобщения, снижения зависимости от данных и обеспечения физически последовательных прогнозов. Основными компонентами моделей PINN являются:
- архитектура нейронной сети - зачастую, глубокая нейронная сеть, которая аппроксимирует решение физической задачи.
- функция потерь на основе физики, которая помимо обычных членов может включать невязки уравнений, описывающих физические процессы (например, уравнений Максвелла, волнового уравнения) или законов сохранения.
- автоматическое дифференцирование, применяемое для вычисления производных выходных данных нейросети относительно входных данных, обеспечивая соблюдение физических ограничений без вычисления конечных разностей.
- гибридное обучение, которое объединяет контролируемое обучение (из разреженных данных) и неконтролируемое обучение (из физических уравнений).

Так, например, если решается уравнение в частных производных $\mathcal{L}(u) = f, \boldsymbol{x}\in\Omega$ с граничными условиями $\mathcal{B}(u)=g, \boldsymbol{x}\in\partial\Omega$, типичная функция потерь для PINN выглядит как 
$$ L = L_{d} + \alpha L_{p} $$
где первое слагаемое обусловлено данными $L_d = || u_{pred} - u_{obs} ||$, а второе - невязаками, связанными с физической задачей
$$ L_p = ||\mathcal{L}(u_{pred}) - f|| + || \mathcal{B}(u_{pred})-g || $$

#### Литература

1. Jiang, J., Chen, M. & Fan, J.A. [Deep neural networks for the evaluation and design of photonic devices](https://www.nature.com/articles/s41578-020-00260-1), Nat. Rev. Mater. 6, 679–700 (2021)
2. Pedro Freire, Egor Manuylovich, Jaroslaw E. Prilepsky, and Sergei K. Turitsyn, [Artificial neural networks for photonic applications—from algorithms to implementation: tutorial](https://opg.optica.org/aop/fulltext.cfm?uri=aop-15-3-739&id=539680), Adv. Opt. Photon. 15, 739-834 (2023)
3. Cuomo, S., Di Cola, V.S., Giampaolo, F. et al. [Scientific Machine Learning Through Physics–Informed Neural Networks: Where we are and What’s Next](link.springer.com/article/10.1007/S10915-022-01939-Z). J Sci Comput 92, 88 (2022)
4. Vlad Medvedev, Andreas Erdmann, and Andreas Rosskopf, [Physics-informed deep learning for 3D modeling of light diffraction from optical metasurfaces](https://opg.optica.org/oe/fulltext.cfm?uri=oe-33-1-1371&id=566869), Opt. Express 33, 1371-1384 (2025)